<a href="https://colab.research.google.com/github/Mal-web-cmd/Python_Libraries/blob/main/ExitAssessment.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import Libraries

In [113]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
import re
import nltk
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout
from tensorflow.keras.layers import Embedding, LSTM, Dense
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report
from sklearn.linear_model import LogisticRegression
import matplotlib.pyplot as plt
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

Reading File path

In [5]:
data=pd.read_csv('/content/tweet_emotions.csv')

Data Preprocessing

In [53]:
data.head()

,1963083266,worry,"@RedFaction The stream is amazing so far. Will it be up for download later, sadly enough, I missed the first hour of it."
0,1963083363,worry,Thunder &amp; lightning scarededededed me!
1,1963083664,sadness,@MsTeagan Sadly cannot listen... *sob*
2,1963083742,worry,@goddessgreeneye i know but it will make us al...
3,1963083775,worry,@sarah6800 why r y depressed? i mite b moving...
4,1963084133,worry,@LIL_MISS_CBW Sorry! It's been one hell of a ...


In [7]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 29010 entries, 0 to 29009
Data columns (total 3 columns):
 #   Column                                                                                                                    Non-Null Count  Dtype 
---  ------                                                                                                                    --------------  ----- 
 0   1963083266                                                                                                                29010 non-null  int64 
 1   worry                                                                                                                     29010 non-null  object
 2   @RedFaction The stream is amazing so far. Will it be up for download later, sadly enough, I missed the first hour of it.  29010 non-null  object
dtypes: int64(1), object(2)
memory usage: 680.1+ KB


In [8]:
data.describe()

,1963083266
count,2.901000e+04
mean,1.801612e+09
std,1.121034e+08
min,1.693956e+09
25%,1.695695e+09
50%,1.752895e+09
75%,1.963711e+09
max,1.966441e+09


Checking for duplicate values

In [15]:
data.duplicated().sum()

0

In [16]:
data.isna().sum()

,0
1963083266,0
worry,0
"@RedFaction The stream is amazing so far. Will it be up for download later, sadly enough, I missed the first hour of it.",0


In [17]:
print(data.isnull().sum())

1963083266                                                                                                                  0
worry                                                                                                                       0
@RedFaction The stream is amazing so far. Will it be up for download later, sadly enough, I missed the first hour of it.    0
dtype: int64


In [52]:
print("Columns in dataset:", data.columns)

Columns in dataset: Index(['1963083266', 'worry',
       '@RedFaction The stream is amazing so far. Will it be up for download later, sadly enough, I missed the first hour of it.'],
      dtype='object')


In [18]:
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [78]:
data = data.rename(columns={'@RedFaction The stream is amazing so far. Will it be up for download later, sadly enough, I missed the first hour of it.': 'content'})
data = data.rename(columns={'worry': 'sentiment'})

In [79]:
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z ]', '', text)  # Remove special characters
def preprocess_text(text):
    text = re.sub(r'http\S+', '', text)  # Remove URLs
    text = re.sub(r'[^a-zA-Z ]', '', text)  # Remove special characters
    text = text.lower()  # Convert to lowercase
    tokens = word_tokenize(text)
    tokens = [word for word in tokens if word not in stopwords.words('english')]
    lemmatizer = WordNetLemmatizer()
    tokens = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(tokens)
    data['cleaned_text'] = data['content'].apply(preprocess_text) # Apply to the 'content' column to generate preprocessed text

In [80]:
X = data['content']
y = data['sentiment']
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

Feature extraction

In [50]:

def extract_features(data, method='tfidf'):
    if method == 'bow':
        vectorizer = CountVectorizer()
    elif method == 'tfidf':
        vectorizer = TfidfVectorizer()
    else:
        raise ValueError("Invalid method. Choose 'bow' or 'tfidf'.")
    features = vectorizer.fit_transform(data)
    return features, vectorizer

Model Training and Evaluation

In [63]:
def train_model(X_train, y_train, X_test, y_test):

    # Initialize the model
    model = MultinomialNB()

    # Train the model on the training data
    model.fit(X_train, y_train)

    # Make predictions on the testing data
    y_pred = model.predict(X_test)

    # Evaluate the model
    accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)

    print(f"Accuracy: {accuracy}")
    print(f"Classification Report:\n{report}")

    return model # Return the trained model

# Call train_model after feature extraction and splitting the data
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Extract features
X_train_features, vectorizer = extract_features(X_train, method='tfidf')
X_test_features = vectorizer.transform(X_test) # Use the same vectorizer for test data

# Train the model and get the trained model back
trained_model = train_model(X_train_features, y_train, X_test_features, y_test)

Accuracy: 0.29696656325405035
Classification Report:
              precision    recall  f1-score   support

       anger       0.00      0.00      0.00        10
     boredom       0.00      0.00      0.00        20
       empty       0.00      0.00      0.00       129
  enthusiasm       0.00      0.00      0.00       109
         fun       0.00      0.00      0.00       304
   happiness       0.37      0.28      0.32       957
        hate       0.00      0.00      0.00       153
        love       0.61      0.16      0.26       710
     neutral       0.27      0.72      0.39      1310
      relief       0.00      0.00      0.00       248
     sadness       0.67      0.00      0.01       568
    surprise       0.00      0.00      0.00       298
       worry       0.28      0.40      0.33       986

    accuracy                           0.30      5802
   macro avg       0.17      0.12      0.10      5802
weighted avg       0.31      0.30      0.23      5802



/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [68]:
model = LogisticRegression()
# Assuming X_train_features from previous cell is the correct feature set
# If not, replace with the correct variable name
model.fit(X_train_features, y_train)

/usr/local/lib/python3.11/dist-packages/sklearn/linear_model/_logistic.py:465: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


LogisticRegression()

In [82]:
tokenizer = Tokenizer(num_words=10000)
# Assuming 'content' is the column with the tweet text
tokenizer.fit_on_texts(data['content'])
sequences = tokenizer.texts_to_sequences(data['content'])

In [106]:
X_train_pad = pad_sequences(X_train_seq, maxlen=max_length, padding='post')
X_test_pad = pad_sequences(X_test_seq, maxlen=max_length, padding='post')

In [115]:
from sklearn.preprocessing import LabelEncoder
# Assuming y_train and y_test are your original target variables
label_encoder = LabelEncoder()
y_train_encoded = label_encoder.fit_transform(y_train) # Fit on training data and transform
y_test_encoded = label_encoder.transform(y_test) # Transform test data using the same encoder


history = model.fit(X_train_pad, y_train_encoded, epochs=5, batch_size=32, validation_data=(X_test_pad, y_test_encoded)) # Use encoded targets

Epoch 1/5


InvalidArgumentError: Graph execution error:

Detected at node compile_loss/sparse_categorical_crossentropy/SparseSoftmaxCrossEntropyWithLogits/SparseSoftmaxCrossEntropyWithLogits defined at (most recent call last):
  File "<frozen runpy>", line 198, in _run_module_as_main

  File "<frozen runpy>", line 88, in _run_code

  File "/usr/local/lib/python3.11/dist-packages/colab_kernel_launcher.py", line 37, in <module>

  File "/usr/local/lib/python3.11/dist-packages/traitlets/config/application.py", line 992, in launch_instance

  File "/usr/local/lib/python3.11/dist-packages/ipykernel/kernelapp.py", line 712, in start

  File "/usr/local/lib/python3.11/dist-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.11/asyncio/base_events.py", line 608, in run_forever

  File "/usr/lib/python3.11/asyncio/base_events.py", line 1936, in _run_once

  File "/usr/lib/python3.11/asyncio/events.py", line 84, in _run

  File "/usr/local/lib/python3.11/dist-packages/ipyker